In [ ]:
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz']
manga_extensions = ['.cbz', '.cbr', '.cb7', '.cbt', '.cbg', '.cbb', '.cbf']

src_ext = img_extensions + zip_extensions + manga_extensions
print(src_ext)

In [ ]:
'''
메인 디렉토리가 /home/manga

/home/manga/(장르 별 하위 디렉토리)/(manga 디렉토리)/(이미지 파일들(압축파일도 있을 수 있음))
예시 
/home/manga/manga/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
/home/manga/__manga2/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
/home/manga/__그림그리기/manga디렉토리/이미지 파일들(압축파일도 있을 수 있음)
등등 여러개 있음

...

manga 확장자는 
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']

manga 압축파일 
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz']

1. 압축파일있는 디렉토리와 image 파일있는 디렉토리는 제외하고 이미지 파일이 없거나 이미지 파일 수가 10개 미만인 디렉토리를 찾아서 하위 디렉토리와 파일들은 /home/manga/__del 디렉토리로 이동하고 삭제
2. 압축파일이있는 디렉토리중 그림파일이 2개 이하면 삭제하고 압축파일에서 한장씩만 문자 안깨지게 UTF-8 인코딩으로 같은 디렉토리에 복사
3. manga 디렉토리 아래 이미지 파일들 숫자.jpg 숫자.webp 형식으로 이름 변경
4. sqlite3 데이터베이스 만들고 디렉토리 정보 저장
    'folder' : manga 디렉토리의 /home/manga/ 부분 제외한 디렉토리 경로
    'images' : manga 디렉토리 아래 이미지 파일 목록
    'zip_files' : manga 디렉토리 아래 압축파일 목록
    'file_date' : 이미지 파일들 생성일자 중 가장 오래된 날짜
    'create_date' : db 생성일자
    'update_date' : db 수정일자
    'tag' : ''
    
    
    
'''


In [1]:
import re, os
filename = 'aaa/몰라_01-001.jpg'
basename = os.path.basename(filename)
print('basename', basename)
name, ext = os.path.splitext(basename)
print('name', name)
print('ext', ext)
number_part = re.findall(r'\d+$', name)[0]  # 뒷부분 숫자 선택
prefix_part = re.findall(r'_?([\d]{2})-', name)[0]  # 앞부분 숫자 선택
title_part = name.replace(prefix_part, '').replace(number_part, '')    # 뒷부분 숫자 제거
print('prefix_part', prefix_part)
print('number_part', number_part)
print('title_part', title_part)


basename 몰라_01-001.jpg
name 몰라_01-001
ext .jpg
prefix_part 01
number_part 001
title_part 몰라_-0


In [51]:
import re, os

def get_volume(text: str) -> str:
    patterns = [
        r'_[^_]+편_\d{1,3}$',
        r'(_\d{1,3}_\d{1,3}_\d{1,3}_\d{1,3}$)',
        r'(_\d{1,3}_\d{1,3}_\d{1,3}$)',
        r'(_\d{1,3}_\d{1,3}$)',
        r'(_[\d]{1,3})$',
        r'_\d{1,3}_무삭제판?$',
        r'_\d{1,3}_[상하]$',
        # r'_(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3})$',
        # r'_(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3}?)_?(\d{1,3})$',
        r'(_[上下상하])_',
        r'(_[上下상하])$',
        # r'(_[상하])_',
        # r'(_[상하])$',
        r'(_[^_]{1,5}[편권판])$',
        r'_[vV][oO][lL][uU]?[mM]?[eE]?(_?\d{1,3})_?',
        r'_[vV][oO][lL][uU]?[mM]?[eE]?(_?\d{1,3})$  ',
        r'_[eE][pP][iI][sS][oO][dD][eE](_?\d{1,3})_?',
        r'_[eE][pP][iI][sS][oO][dD][eE](_?\d{1,3})$',
        r'_[eE][pP](_?\d{1,3})_?',
        r'_[eE][pP](_?\d{1,3})$',
        r'(_\d{1,3})_',
        r'(_\d{1,3})$',
        r'_[sS][eE][rR][iI][eE][sS](_?\d{1,3})_?',
        r'_[sS][eE][rR][iI][eE][sS](_?\d{1,3})$',
        r'_[pP][aA][rR][tT](_?\d{1,3})_?',
        r'_[pP][aA][rR][tT](_?\d{1,3})$',
        r'_[cC][hH]{0,2}(_?\d{1,3}_?\d{1,3})_?',
        r'(_제?\d{1,3}화)_?',
        r'(_\d{1,3}회)_?',
        r'_[sS][eE][rR][iI][eE][sS]_[cC](_\d{1,3})$',
        r'_\d{1,3}_?무삭제',
        r'_1st$',
        r'_2nd$',
        r'_3rd$',
    ]
    for p in patterns:
        result = re.findall(p, text)
        if result:
            return result
    return ''
        

def get_eng_title(text: str) -> str:

    patterns = [
        r'[가-힣]+_\d+$',
        r'[가-힣]_?\d+_?[가-힣]',
        r'[가-힣]+_',
        r'[^a-zA-Z0-9]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_kor_title(text: str) -> str:
    patterns = [
        r'[^가-힣0-9]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_jpn_title(text: str) -> str:
    patterns = [
        r'[^0-9ぁ-んァ-ヶ\u3400-\u4dbf\u4e00-\u9fff0-9]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_eng_kor_title(text: str) -> str:
    patterns = [
        r'[^a-zA-Z0-9가-힣]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_eng_jpn_title(text: str) -> str:
    patterns = [
        r'[^a-zA-Z0-9\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_kor_jpn_title(text: str) -> str:
    patterns = [
        r'[^가-힣0-9\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_eng_kor_jpn_title(text: str) -> str:
    patterns = [
        r'[^a-zA-Z0-9\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff가-힣]',
        r'[\s]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    
    return text.strip().replace(' ', '_')

def get_only_kor_first_title(text: str) -> str:
        patterns = [
            r'[^가-힣]',
            r'[\s]+'
        ]
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        
        return text.strip().split(' ')[0]

def get_title(text: str) -> str:
    texts = {
        'original': text,
        'eng': get_eng_title(text), 
        'kor': get_kor_title(text), 
        'jpn': get_jpn_title(text), 
        'eng_kor': get_eng_kor_title(text), 
        'eng_jpn': get_eng_jpn_title(text), 
        'kor_jpn': get_kor_jpn_title(text), 
        'eng_kor_jpn': get_eng_kor_jpn_title(text),
    }
    # texts['only_eng'] = texts['eng_kor'].split('_'+get_only_kor_first_title(text))[0]
    # # print("texts['only_eng']:", texts['only_eng'])
    words_cnt = [texts[t] for t in texts if len(texts[t].split('_')) > 1]
    words_cnt.sort(key=lambda x: len(x.split('_')))
    # print(words_cnt)
    _length = [len(texts[t]) for t in texts if len(texts[t]) > 0]
    max_length = max(_length)
    min_length = min(_length)
    max_text = [texts[t] for t in texts if len(texts[t]) == max_length][0]
    min_text = [texts[t] for t in texts if len(texts[t]) == min_length][0]
    
    if len(texts['eng_kor_jpn']) < 31:
        return texts['eng_kor_jpn']
    elif len(texts['eng'].split('_')) > 3:
        return texts['eng']
    elif texts['kor']:
        return texts['kor']
    elif texts['jpn']:
        return texts['jpn']
    else:
        return min_text
    
        
base_path = '/home/manga/manga3'
# for sub_path in os.listdir(base_path):
#     # print(sub_path)
#     if get_volume(sub_path):
#         print(get_volume(sub_path)[1].ljust(10, ' '), get_volume(sub_path)[0].ljust(30, ' '), sub_path)

for sub_path in os.listdir(base_path):
    if os.path.isdir(os.path.join(base_path, sub_path)):# and len(get_title(sub_path)) >3:
        # print(sub_path)
        try:
            patterns = [
                r'_?korean_팀_[가-힣]+$',
                r'_korean$',
            ]  
            for pattern in patterns:
                sub_path = re.sub(pattern, '', sub_path)    
            if get_volume(sub_path):    
                print(get_title(sub_path),'|', f"==>{get_volume(sub_path)[0]}<==", sub_path)
                print("==============================")
            else:
                print(get_title(sub_path), '|', sub_path)
        except Exception as e:
            print(e, sub_path, '+++++++++++++++++++++++++++++++++++++++++++')
        


츄_브르즈 | 츄_브르즈
치녀_풍기_위원의_모두에게_말할_수_없는_부탁_2 | ==>_2<== 치녀_풍기_위원의_모두에게_말할_수_없는_부탁_2
친구_밀애 | 친구_밀애
친누나한테_펨돔조교_당하는_만화 | 친누나한테_펨돔조교_당하는_만화
친절한_간호사_키리시마_씨 | 친절한_간호사_키리시마_씨
친절한_빛과_색 | 친절한_빛과_색
친절한_이웃누나 | 친절한_이웃누나
카리나의_비밀상점_Part_2 | ==>_2<== 카리나의_비밀상점_Part_2
카미사토_아야카 | 카미사토_아야카
카미사토_아야카_Another_story | 카미사토_아야카_Another_story
카야네토리_카야누나_시리즈_애장판 | ==>_애장판<== 카야네토리_카야누나_시리즈_애장판
카이사_조교일기 | 카이사_조교일기
카쿠다테_카린 | 카쿠다테_카린
카타리나_가렌 | 카타리나_가렌
캠퍼스_난교 | 캠퍼스_난교
캠퍼스_난교_2 | ==>_2<== 캠퍼스_난교_2
케틀린 | 케틀린
코스프레 | 코스프레
코쿠하레_1 | ==>_1<== 코쿠하레_1
코쿠하레_2 | ==>_2<== 코쿠하레_2
코쿠하레_3 | ==>_3<== 코쿠하레_3
코쿠하레_4 | ==>_4<== 코쿠하레_4
코쿠하레_5 | ==>_5<== 코쿠하레_5
콜걸_아가씨는_질투_따위_안_해 | 콜걸_아가씨는_질투_따위_안_해
쾌락의_문 | 쾌락의_문
쿨_시크한_내_여친은_언제나_포커페이스 | 쿨_시크한_내_여친은_언제나_포커페이스
큰누나의_애기엄마_친구 | 큰누나의_애기엄마_친구
큰함마_단편선_소녀전선_느그형의_방문 | 큰함마_단편선_소녀전선_느그형의_방문
클럽_난교 | 클럽_난교
키리코_키리코맘_갱뱅 | 키리코_키리코맘_갱뱅
킨드레드 | 킨드레드
탐이_많은_이모 | 탐이_많은_이모
터치미 | 터치미
토끼상사_망가 | 토끼상사_망가
토라코랑_with_torako | 토라코랑_with_torako
토벌_동정_사냥꾼_킬러 | 토벌_동정_사냥꾼_킬러
토우코_선배와_구교사에서 | 토우코_선배와_구교사에서
파블로프의_암캐 | 파블로프의_암캐

In [ ]:
{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f9ac9a75670>, 
 'height': 720, 
 'face': None, 
 'pregnant': None, 
 'uniform': None, 
 'showtime': 577, 
 'look': None, 
 'conversation': None, 
 'family': None, 
 'bitrate': 345561, 
 'age': None, 
 'lesbian': None, 
 'ad_start': None, 
 'id': 7926, 
 'filesize': 24922302, 
 'pussy': None, 
 'ani': None, 
 'ad_finish': None, 
 'cdate': datetime.datetime(2022, 6, 25, 10, 40, 49), 
 'etc': None, 'oral': None, 'star': None, 'display_quality': None, 
 'school_uniform': None, 'masturbation': None, 'date_posted': datetime.date(2022, 6, 27), 
 'dbid': 'test2/나스 왜 써요- 시놀로지 나스(NAS) 개념부터 구글 드라이브 비교 장단점, 나스 선택법까지 한 번에! - 나도 샀다 NAS EP01.mp4', 
 'country': None, 
 'hip': None, 
 'massage': None, 
 'date_modified': datetime.date(2022, 7, 21), 
 'width': 1280, 
 'group': None}

In [5]:
import re, json

text = "file_1편 file_2편 file_3편"

# 1. search(): 첫 번째 매치만 찾음
search_result = re.search(r'_(\d+)편', text)
if search_result:
    print("search 결과:", search_result.group(1))  # "1"
    print("위치:", search_result.span())           # 매치된 위치
    print("전체 매치:", search_result.group(0))    # 전체 매치 문자열
    
search_result = re.findall(r'_(\d+)편', text)
print(search_result)


t = 'abc_34a_fd'
if 'c_3' in t:
    print('ok')
    print(t[:50])
a = []
print(a[:-1])


etc= '{"etc":"sod,sdjs,sdmu"}'
etc = json.loads(etc)
print(type(etc))
sub = '#쿼리#은아'
c = r'^#[a-zA-Z0-9가-힣]{2,5}+#'
_subject = re.sub(c, '', sub)
print(_subject)

search 결과: 1
위치: (4, 7)
전체 매치: _1편
['1', '2', '3']
ok
abc_34a_fd
[]
<class 'dict'>
은아


In [ ]:
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK

INFO:     192.168.0.1:0 - "GET /video/2671/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13381/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/5854/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10985/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13548/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/4533/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/44/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6928/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14171/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13492/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14217/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2100/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11654/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1238/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/824/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9557/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11800/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1349/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6268/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2094/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6972/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1679/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11894/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/3001/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14313/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2173/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/7475/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10207/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2884/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13381/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2671/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14335/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14335/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/5854/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/7459/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13830/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9972/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13346/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9486/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/4352/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13941/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1785/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14223/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11252/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/82/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/8199/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10208/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11985/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9240/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1707/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6082/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/549/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14171/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13492/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1238/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/824/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9400/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9557/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/57/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10064/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/4572/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/8930/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14519/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/7360/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2652/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9429/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/5013/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10740/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10985/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13548/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/4533/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/44/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6928/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14217/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2100/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10901/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9486/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/7459/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1785/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13830/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9972/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13346/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/4352/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/13941/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/82/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/8199/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10208/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9240/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11985/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14223/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11800/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10901/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1707/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6082/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/549/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11252/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1349/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6268/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/57/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2094/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/6972/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/1679/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10064/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/3001/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14313/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10207/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11894/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2173/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/7475/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2884/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/4572/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/8930/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/7360/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14519/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/2652/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9429/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/5013/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/10740/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/9400/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/11654/rating HTTP/1.0" 200 OK










In [ ]:
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14509/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14524/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14509/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14524/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14503/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14504/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14505/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14506/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14507/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14510/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14508/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14512/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14513/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14511/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14514/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14516/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14517/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14518/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14519/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14520/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14515/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14521/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14522/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14523/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14525/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14526/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14527/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14528/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14529/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14530/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14531/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14532/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14503/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14504/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14505/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14506/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14507/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14509/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14508/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14510/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14524/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14512/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14513/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14511/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14514/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14515/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14522/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14523/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14527/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14516/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14518/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14517/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14519/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14521/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14520/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14525/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14528/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14526/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14529/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14530/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14532/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14516/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14511/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14531/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14504/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14503/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14517/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14519/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14520/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14518/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14505/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14505/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14519/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14517/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14520/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14518/rating HTTP/1.0" 200 OK
INFO:     192.168.0.43:0 - "GET /auth/nginxauth HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14506/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14507/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14522/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14527/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14523/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14515/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14525/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14521/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14509/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14510/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14508/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14528/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14524/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14526/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14513/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14512/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14514/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14529/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14532/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14530/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14511/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14516/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14531/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14504/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14503/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14506/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14507/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14522/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14527/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14523/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14515/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14525/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14521/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14510/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14508/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14528/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14526/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14512/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14513/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14532/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14529/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14514/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14530/rating HTTP/1.0" 200 OK
INFO:     192.168.0.1:0 - "GET /video/14531/rating HTTP/1.0" 200 OK

In [6]:
import os
import shutil
import sqlite3
from datetime import datetime
import pytz
import zipfile
import py7zr
import rarfile
import patoolib
from pathlib import Path

# 한국 시간대 설정
KST = pytz.timezone('Asia/Seoul')

# 파일 확장자 정의
img_extensions = ['.jpg', '.png', '.jpeg', '.webp', '.gif', '.bmp', '.tiff', '.ico', '.svg']
zip_extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz', '.alz']

def get_file_list(directory):
    """디렉토리 내의 이미지와 압축 파일 목록을 반환"""
    images = []
    zip_files = []
    
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            ext = os.path.splitext(file)[1].lower()
            if ext in img_extensions:
                images.append(file_path)
            elif ext in zip_extensions:
                zip_files.append(file_path)
                
    return images, zip_files

def process_manga_directory(manga_path):
    """개별 만화 디렉토리 처리"""
    images, zip_files = get_file_list(manga_path)
    
    # 조건 1: 이미지가 없거나 10개 미만이고 압축파일이 없는 경우
    if len(zip_files) == 0 and (len(images) == 0 or len(images) < 10):
        del_path = os.path.join('/home/manga/__del', os.path.relpath(manga_path, '/home/manga'))
        os.makedirs(os.path.dirname(del_path), exist_ok=True)
        try:
            shutil.move(manga_path, del_path)
        except Exception as e:
            print(f"Error moving {manga_path} to {del_path}: {str(e)}")
        return
    
    # 조건 2: 압축파일이 있고 이미지가 2개 이하인 경우
    if zip_files and len(images) <= 2:
        for img in images:
            os.remove(img)
        
        for zip_file in zip_files:
            try:
                extract_first_image(zip_file, manga_path)
            except Exception as e:
                print(f"Error processing {zip_file}: {str(e)}")
    
    # 조건 3: 이미지 파일 이름 변경
    rename_images(manga_path)

def extract_first_image(zip_file, dest_dir):
    """압축 파일에서 첫 번째 이미지 추출"""
    ext = os.path.splitext(zip_file)[1].lower()
    
    try:
        if ext == '.zip':
            with zipfile.ZipFile(zip_file, 'r') as zf:
                for file in zf.namelist():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        zf.extract(file, dest_dir)
                        break
        elif ext == '.7z':
            with py7zr.SevenZipFile(zip_file, 'r') as sz:
                for file in sz.getnames():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        sz.extract(dest_dir, [file])
                        break
        elif ext == '.rar':
            with rarfile.RarFile(zip_file, 'r') as rf:
                for file in rf.namelist():
                    if any(file.lower().endswith(ext) for ext in img_extensions):
                        rf.extract(file, dest_dir)
                        break
        else:
            patoolib.extract_archive(zip_file, outdir=dest_dir, interactive=False)
    except Exception as e:
        print(f"Error extracting {zip_file}: {str(e)}")

def rename_images(directory):
    """이미지 파일 이름을 숫자 형식으로 변경"""
    images = [f for f in os.listdir(directory) if any(f.lower().endswith(ext) for ext in img_extensions)]
    images.sort()
    
    for i, img in enumerate(images, 1):
        ext = os.path.splitext(img)[1].lower()
        if ext == '.webp':
            new_name = f"{i}.webp"
        else:
            new_name = f"{i}.jpg"
        old_path = os.path.join(directory, img)
        new_path = os.path.join(directory, new_name)
        os.rename(old_path, new_path)

def update_database(manga_path):
    """SQLite 데이터베이스 업데이트"""
    conn = sqlite3.connect('manga.db')
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS manga_dirs (
        folder TEXT PRIMARY KEY,
        images TEXT,
        zip_files TEXT,
        file_date TEXT,
        create_date TEXT,
        update_date TEXT,
        tag TEXT
    )
    ''')
    
    images, zip_files = get_file_list(manga_path)
    relative_path = os.path.relpath(manga_path, '/home/manga')
    
    if images:
        file_date = min(datetime.fromtimestamp(os.path.getmtime(img), KST) 
                      for img in images).strftime('%Y-%m-%d %H:%M:%S')
    else:
        file_date = ''
        
    now = datetime.now(KST).strftime('%Y-%m-%d %H:%M:%S')
    
    cursor.execute('''
    INSERT OR REPLACE INTO manga_dirs 
    (folder, images, zip_files, file_date, create_date, update_date, tag)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        relative_path,
        ','.join([os.path.basename(img) for img in images]),
        ','.join([os.path.basename(zip_f) for zip_f in zip_files]),
        file_date,
        now,
        now,
        ''
    ))
    
    conn.commit()
    conn.close()

def main():
    manga_root = '/home/manga'
    
    # 모든 장르 디렉토리 순회
    for genre_dir in os.listdir(manga_root):
        genre_path = os.path.join(manga_root, genre_dir)
        if not os.path.isdir(genre_path):
            continue
            
        # 각 장르 디렉토리 내의 만화 디렉토리 순회
        for manga_dir in os.listdir(genre_path):
            manga_path = os.path.join(genre_path, manga_dir)
            if not os.path.isdir(manga_path):
                continue
                
            # 만화 디렉토리 처리
            process_manga_directory(manga_path)
            
            # 데이터베이스 업데이트
            if os.path.exists(manga_path):  # 삭제되지 않은 경우만
                update_database(manga_path)

if __name__ == "__main__":
    main()

Error moving /home/manga/__del/__del to /home/manga/__del/__del/__del: Cannot move a directory '/home/manga/__del/__del' into itself '/home/manga/__del/__del/__del'.
Error extracting /home/manga/__manga2/神坂公平/번역_神坂公平_sisters～여름의_마지막_날～.rar: Cannot find working tool
Error extracting /home/manga/__manga2/第6基地/번역_C80_第6基地_나코_라이프_꽃이_피는_첫걸음_동인지.rar: Cannot find working tool
Error extracting /home/manga/__manga2/菊一もんじ/번역_菊一もんじ_アナリスト.rar: Cannot find working tool
Error extracting /home/manga/__manga2/藤原俊一/팀H번역_藤原俊一_에프터_스쿨_상업지.rar: Cannot find working tool
Error extracting /home/manga/__manga2/軒下の猫屋/번역_C83_軒下の猫屋_アルデヒド_어둠의_불꽃에_안기고_싶어_중2병이라도_사랑이_하고_싶어_팀아키바.rar: Cannot find working tool


### video 폴더
/home/video/


In [4]:
pip install pytz py7zr rarfile patool

nginx service is running...
code service is running...
Note: you may need to restart the kernel to use updated packages.
